In [1]:

import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from bs4 import BeautifulSoup as bs

In [2]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# !conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests

# !conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library


Libraries imported.


In [3]:
df_toronto = pd.read_csv('toronto_base.csv')
df_toronto.head(7)

,Unnamed: 0,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,0,M3A,North York,Parkwoods,43.753259,-79.329656
1,1,M4A,North York,Victoria Village,43.725882,-79.315572
2,2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
5,5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353


### Create Map For Toronto:

In [4]:

# for the city Toronto, latitude and longtitude are manually extracted via google search
toronto_latitude = 43.6532; toronto_longitude = -79.3832
map_toronto = folium.Map(location = [toronto_latitude, toronto_longitude], zoom_start = 10.7)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    

map_toronto.save("my_map1.html" )

In [5]:

from IPython.display import HTML

HTML('<iframe src=my_map1.html width=700 height=450></iframe>')

C:\Users\anushka ambavkar\anaconda3\lib\site-packages\IPython\core\display.py:701: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


In [6]:
# @hiddel_cell
CLIENT_ID = 'FPIENWUBAFUUMXHPL5QK0LZICLXB1YAPUAHHPQCPVU3CUJB2' # your Foursquare ID
CLIENT_SECRET = 'CJBSSJYWDAUKOG40C5CFOJVYVILMAHI2AYPNBZGUWDX0Y1VJ' # your Foursquare Secret
VERSION = '20190530' # Foursquare API version

In [7]:

scarborough_data = df_toronto[df_toronto['Borough'] == 'Scarborough'].reset_index(drop=True)
scarborough_data.head(7)

,Unnamed: 0,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,12,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,18,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,22,M1G,Scarborough,Woburn,43.770992,-79.216917
4,26,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,32,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,38,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029


### Create a map of Scarborough and its neighbourhoods

In [8]:
address_scar = 'Scarborough,Toronto'
latitude_scar = 43.773077
longitude_scar = -79.257774
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude_scar, longitude_scar))

The geograpical coordinate of Scarborough are 43.773077, -79.257774.


In [9]:
map_scarb = folium.Map(location=[latitude_scar, longitude_scar], zoom_start=12)

# add markers to map
for lat, lng, label in zip(scarborough_data['Latitude'], scarborough_data['Longitude'], scarborough_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_scarb)  
    
map_scarb.save('my_map2.html')

In [10]:
from IPython.display import HTML

HTML('<iframe src=my_map2.html width=700 height=450></iframe>')

C:\Users\anushka ambavkar\anaconda3\lib\site-packages\IPython\core\display.py:701: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


In [11]:
neighborhood_latitude = scarborough_data.loc[0, 'Latitude'] # neighbourhood latitude value
neighborhood_longitude = scarborough_data.loc[0, 'Longitude'] # neighbourhood longitude value

neighborhood_name = scarborough_data.loc[0, 'Neighbourhood'] # neighbourhood name

print('Latitude and longitude values of "{}" are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of "Malvern, Rouge" are 43.8066863, -79.19435340000003.


In [12]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_scar, longitude_scar, VERSION, radius, LIMIT)

In [13]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '6017eea93415ad6d97bebdc9'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Scarborough City Centre',
  'headerFullLocation': 'Scarborough City Centre, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 42,
  'suggestedBounds': {'ne': {'lat': 43.7775770045, 'lng': -79.25155367954714},
   'sw': {'lat': 43.7685769955, 'lng': -79.26399432045285}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5085ec39e4b0b1ead2eb0818',
       'name': 'Disney Store',
       'location': {'address': '300 Borough Drive',
        'crossStreet': 'in Scarborough Town Centre',
        'lat': 43.775537,
        'lng': -79.256833,
        'labeledLa

In [14]:

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [15]:
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

venues = results['response']['groups'][0]['items']  
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(10)

C:\Users\anushka ambavkar\anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  after removing the cwd from sys.path.


,name,categories,lat,lng
0,Disney Store,Toy / Game Store,43.775537,-79.256833
1,SEPHORA,Cosmetics Shop,43.775017,-79.258109
2,Tommy Hilfiger,Clothing Store,43.776015,-79.257369
3,American Eagle Store,Clothing Store,43.776012,-79.258334
4,Hot Topic,Clothing Store,43.775450,-79.257929
5,Coliseum Scarborough Cinemas,Movie Theater,43.775995,-79.255649
6,Chipotle Mexican Grill,Mexican Restaurant,43.776410,-79.258069
7,St. Andrews Fish & Chips,Fish & Chips Shop,43.771865,-79.252645
8,Shoppers Drug Mart,Pharmacy,43.773305,-79.251662
9,Tim Hortons,Coffee Shop,43.769936,-79.260368


In [16]:

print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

42 venues were returned by Foursquare.


In [17]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Get venues for each neighborhood in Scarborough

In [18]:
scarborough_venues = getNearbyVenues(names=scarborough_data['Neighbourhood'],
                                   latitudes=scarborough_data['Latitude'],
                                   longitudes=scarborough_data['Longitude']
                                  )

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge


In [19]:

scarborough_venues.head(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Sail Sushi,43.765951,-79.191275,Restaurant
5,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant
6,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Enterprise Rent-A-Car,43.764076,-79.193406,Rental Car Location
7,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Woburn Medical Centre,43.766631,-79.192286,Medical Center
8,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Lawrence Ave E & Kingston Rd,43.767704,-79.189490,Intersection
9,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Eggsmart,43.767800,-79.190466,Breakfast Spot


In [20]:

scarborough_venues.tail(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
82,"Steeles West, L'Amoreaux West",43.799525,-79.318389,Shoppers Drug Mart,43.799966,-79.317985,Pharmacy
83,"Steeles West, L'Amoreaux West",43.799525,-79.318389,Pizza Pizza,43.797909,-79.318113,Pizza Place
84,"Steeles West, L'Amoreaux West",43.799525,-79.318389,Metro,43.797830,-79.318492,Supermarket
85,"Steeles West, L'Amoreaux West",43.799525,-79.318389,Eggsmart,43.796375,-79.318681,Breakfast Spot
86,"Steeles West, L'Amoreaux West",43.799525,-79.318389,Dollarama,43.798394,-79.318453,Discount Store
87,"Steeles West, L'Amoreaux West",43.799525,-79.318389,Presotea,43.799397,-79.319014,Bubble Tea Shop
88,"Steeles West, L'Amoreaux West",43.799525,-79.318389,Fit4Less,43.798394,-79.318453,Gym
89,"Steeles West, L'Amoreaux West",43.799525,-79.318389,Super Taste Noodle House,43.798217,-79.318513,Noodle House
90,"Steeles West, L'Amoreaux West",43.799525,-79.318389,Nantha's Bakery,43.796430,-79.319151,Indian Restaurant
91,"Steeles West, L'Amoreaux West",43.799525,-79.318389,Ocean Nails Spa,43.795290,-79.320101,Nail Salon


In [21]:
scarborough_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,3,3,3,3,3,3
"Birch Cliff, Cliffside West",5,5,5,5,5,5
Cedarbrae,8,8,8,8,8,8
"Clarks Corners, Tam O'Shanter, Sullivan",13,13,13,13,13,13
"Cliffside, Cliffcrest, Scarborough Village West",2,2,2,2,2,2
"Dorset Park, Wexford Heights, Scarborough Town Centre",6,6,6,6,6,6
"Golden Mile, Clairlea, Oakridge",9,9,9,9,9,9
"Guildwood, Morningside, West Hill",8,8,8,8,8,8
"Kennedy Park, Ionview, East Birchmount Park",6,6,6,6,6,6


In [22]:
print('There are {} uniques categories.'.format(len(scarborough_venues['Venue Category'].unique())))

There are 57 uniques categories.


In [23]:
# one hot encoding
scarb_onehot = pd.get_dummies(scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scarb_onehot['Neighborhood'] = scarborough_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [scarb_onehot.columns[-1]] + list(scarb_onehot.columns[:-1])
scarb_onehot = scarb_onehot[fixed_columns]

scarb_onehot.head()

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bubble Tea Shop,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Convenience Store,Department Store,Discount Store,Electronics Store,Farm,Fast Food Restaurant,Fried Chicken Joint,Furniture / Home Store,Gas Station,General Entertainment,Grocery Store,Gym,Hakka Restaurant,Hobby Shop,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Jewelry Store,Korean BBQ Restaurant,Latin American Restaurant,Lounge,Medical Center,Mexican Restaurant,Middle Eastern Restaurant,Motel,Nail Salon,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Restaurant,Sandwich Place,Skating Rink,Smoke Shop,Soccer Field,Supermarket,Thai Restaurant,Vietnamese Restaurant
0,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Guildwood, Morningside, West Hill",0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


In [24]:

scarb_onehot.shape

(92, 58)

In [25]:
scarb_grouped = scarb_onehot.groupby('Neighborhood').mean().reset_index()
scarb_grouped.head(7)

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bubble Tea Shop,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Convenience Store,Department Store,Discount Store,Electronics Store,Farm,Fast Food Restaurant,Fried Chicken Joint,Furniture / Home Store,Gas Station,General Entertainment,Grocery Store,Gym,Hakka Restaurant,Hobby Shop,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Jewelry Store,Korean BBQ Restaurant,Latin American Restaurant,Lounge,Medical Center,Mexican Restaurant,Middle Eastern Restaurant,Motel,Nail Salon,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Restaurant,Sandwich Place,Skating Rink,Smoke Shop,Soccer Field,Supermarket,Thai Restaurant,Vietnamese Restaurant
0,Agincourt,0.0,0.000,0.0,0.000000,0.000000,0.0,0.333333,0.0,0.000000,0.000000,0.0,0.000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.333333,0.333333,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000
1,"Birch Cliff, Cliffside West",0.0,0.000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.2,0.000,0.000000,0.0,0.2,0.000000,0.0,0.0,0.0,0.2,0.000000,0.000000,0.000000,0.000000,0.2,0.0,0.0,0.000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.2,0.0,0.000000,0.0,0.000000,0.000000
2,Cedarbrae,0.0,0.125,0.0,0.125000,0.125000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.125,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.125000,0.000000,0.125000,0.0,0.0,0.0,0.125,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.125000,0.000000
3,"Clarks Corners, Tam O'Shanter, Sullivan",0.0,0.000,0.0,0.000000,0.076923,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000,0.076923,0.0,0.0,0.076923,0.0,0.0,0.0,0.0,0.153846,0.076923,0.000000,0.076923,0.0,0.0,0.0,0.000,0.0,0.000000,0.000000,0.000000,0.076923,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.076923,0.000000,0.000000,0.076923,0.153846,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.076923,0.000000
4,"Cliffside, Cliffcrest, Scarborough Village West",0.5,0.000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.5,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000
5,"Dorset Park, Wexford Heights, Scarborough Town...",0.0,0.000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000,0.166667,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.166667,0.000000,0.0,0.0,0.0,0.000,0.0,0.000000,0.333333,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.166667,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.166667
6,"Golden Mile, Clairlea, Oakridge",0.0,0.000,0.0,0.222222,0.000000,0.0,0.000000,0.0,0.222222,0.111111,0.0,0.000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000,0.0,0.111111,0.000000,0.111111,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.111111,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.111111,0.0,0.000000,0.000000



### Get top 10 venues per neighborhood

In [26]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [27]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scarb_grouped['Neighborhood']

for ind in np.arange(scarb_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarb_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Breakfast Spot,Latin American Restaurant,Lounge,Vietnamese Restaurant,College Stadium,General Entertainment,Gas Station,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant
1,"Birch Cliff, Cliffside West",College Stadium,General Entertainment,Skating Rink,Farm,Café,Vietnamese Restaurant,Grocery Store,Gas Station,Furniture / Home Store,Fried Chicken Joint
2,Cedarbrae,Thai Restaurant,Bank,Hakka Restaurant,Caribbean Restaurant,Fried Chicken Joint,Gas Station,Bakery,Athletics & Sports,Grocery Store,General Entertainment
3,"Clarks Corners, Tam O'Shanter, Sullivan",Fast Food Restaurant,Pizza Place,Convenience Store,Gas Station,Thai Restaurant,Chinese Restaurant,Bank,Fried Chicken Joint,Italian Restaurant,Pharmacy
4,"Cliffside, Cliffcrest, Scarborough Village West",American Restaurant,Motel,College Stadium,Grocery Store,General Entertainment,Gas Station,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Farm
5,"Dorset Park, Wexford Heights, Scarborough Town...",Indian Restaurant,Vietnamese Restaurant,Pet Store,Furniture / Home Store,Chinese Restaurant,Bank,Bar,General Entertainment,Athletics & Sports,Gas Station
6,"Golden Mile, Clairlea, Oakridge",Bakery,Bus Line,Bus Station,Soccer Field,Ice Cream Shop,Intersection,Park,Vietnamese Restaurant,Discount Store,Electronics Store
7,"Guildwood, Morningside, West Hill",Rental Car Location,Restaurant,Electronics Store,Intersection,Breakfast Spot,Mexican Restaurant,Medical Center,Bank,Fast Food Restaurant,Fried Chicken Joint
8,"Kennedy Park, Ionview, East Birchmount Park",Hobby Shop,Chinese Restaurant,Discount Store,Department Store,Convenience Store,Coffee Shop,Bar,Breakfast Spot,Grocery Store,General Entertainment
9,"Malvern, Rouge",Fast Food Restaurant,Vietnamese Restaurant,Hakka Restaurant,Grocery Store,General Entertainment,Gas Station,Furniture / Home Store,Fried Chicken Joint,Farm,Electronics Store


### Run k-means to cluster the neighborhoods into 5 clusters

In [28]:

# import k-means from clustering stage
from sklearn.cluster import KMeans

scarb_data = scarborough_data.drop(14)
# set number of clusters
kclusters = 5

scarb_grouped_clustering = scarb_grouped.drop('Neighborhood', 1)


# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scarb_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 
#len(kmeans.labels_)#=16
#scarborough_data.shape

array([1, 1, 1, 1, 0, 1, 1, 1, 1, 4])

### Include kmeans.labels_ into the original Scarborough dataframe

In [29]:

scarb_merged = scarb_data

# add clustering labels
scarb_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
scarb_merged = scarb_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

scarb_merged

,Unnamed: 0,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,1,Fast Food Restaurant,Vietnamese Restaurant,Hakka Restaurant,Grocery Store,General Entertainment,Gas Station,Furniture / Home Store,Fried Chicken Joint,Farm,Electronics Store
1,12,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,1,Bar,Vietnamese Restaurant,College Stadium,Grocery Store,General Entertainment,Gas Station,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Farm
2,18,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,1,Rental Car Location,Restaurant,Electronics Store,Intersection,Breakfast Spot,Mexican Restaurant,Medical Center,Bank,Fast Food Restaurant,Fried Chicken Joint
3,22,M1G,Scarborough,Woburn,43.770992,-79.216917,1,Coffee Shop,Indian Restaurant,Korean BBQ Restaurant,College Stadium,General Entertainment,Gas Station,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Farm
4,26,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0,Thai Restaurant,Bank,Hakka Restaurant,Caribbean Restaurant,Fried Chicken Joint,Gas Station,Bakery,Athletics & Sports,Grocery Store,General Entertainment
5,32,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,1,Grocery Store,Jewelry Store,Playground,Coffee Shop,General Entertainment,Gas Station,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Farm
6,38,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029,1,Hobby Shop,Chinese Restaurant,Discount Store,Department Store,Convenience Store,Coffee Shop,Bar,Breakfast Spot,Grocery Store,General Entertainment
7,44,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577,1,Bakery,Bus Line,Bus Station,Soccer Field,Ice Cream Shop,Intersection,Park,Vietnamese Restaurant,Discount Store,Electronics Store
8,51,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476,1,American Restaurant,Motel,College Stadium,Grocery Store,General Entertainment,Gas Station,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Farm
9,58,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848,4,College Stadium,General Entertainment,Skating Rink,Farm,Café,Vietnamese Restaurant,Grocery Store,Gas Station,Furniture / Home Store,Fried Chicken Joint


### Visualize the clusters in the map

In [30]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location = [latitude_scar, longitude_scar], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(scarb_merged['Latitude'], scarb_merged['Longitude'], scarb_merged['Neighbourhood'], scarb_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters.save('my_map3.html')

In [31]:
from IPython.display import HTML

HTML('<iframe src=my_map3.html width=700 height=450></iframe>')

C:\Users\anushka ambavkar\anaconda3\lib\site-packages\IPython\core\display.py:701: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")



### Examine each of the five clusters

In [32]:

scarb_merged.loc[scarb_merged['Cluster Labels'] == 0, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,Postal Code,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,M1H,-79.239476,0,Thai Restaurant,Bank,Hakka Restaurant,Caribbean Restaurant,Fried Chicken Joint,Gas Station,Bakery,Athletics & Sports,Grocery Store,General Entertainment


In [33]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 1, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,Postal Code,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,-79.194353,1,Fast Food Restaurant,Vietnamese Restaurant,Hakka Restaurant,Grocery Store,General Entertainment,Gas Station,Furniture / Home Store,Fried Chicken Joint,Farm,Electronics Store
1,M1C,-79.160497,1,Bar,Vietnamese Restaurant,College Stadium,Grocery Store,General Entertainment,Gas Station,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Farm
2,M1E,-79.188711,1,Rental Car Location,Restaurant,Electronics Store,Intersection,Breakfast Spot,Mexican Restaurant,Medical Center,Bank,Fast Food Restaurant,Fried Chicken Joint
3,M1G,-79.216917,1,Coffee Shop,Indian Restaurant,Korean BBQ Restaurant,College Stadium,General Entertainment,Gas Station,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Farm
5,M1J,-79.239476,1,Grocery Store,Jewelry Store,Playground,Coffee Shop,General Entertainment,Gas Station,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Farm
6,M1K,-79.262029,1,Hobby Shop,Chinese Restaurant,Discount Store,Department Store,Convenience Store,Coffee Shop,Bar,Breakfast Spot,Grocery Store,General Entertainment
7,M1L,-79.284577,1,Bakery,Bus Line,Bus Station,Soccer Field,Ice Cream Shop,Intersection,Park,Vietnamese Restaurant,Discount Store,Electronics Store
8,M1M,-79.239476,1,American Restaurant,Motel,College Stadium,Grocery Store,General Entertainment,Gas Station,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Farm
10,M1P,-79.273304,1,Indian Restaurant,Vietnamese Restaurant,Pet Store,Furniture / Home Store,Chinese Restaurant,Bank,Bar,General Entertainment,Athletics & Sports,Gas Station
12,M1S,-79.262029,1,Breakfast Spot,Latin American Restaurant,Lounge,Vietnamese Restaurant,College Stadium,General Entertainment,Gas Station,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant


In [34]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 2, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,Postal Code,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,M1R,-79.295849,2,Auto Garage,Smoke Shop,Bakery,Sandwich Place,Middle Eastern Restaurant,Vietnamese Restaurant,Farm,Convenience Store,Department Store,Discount Store


In [35]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 3, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,Postal Code,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,M1X,-79.205636,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 4, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,Postal Code,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,M1N,-79.264848,4,College Stadium,General Entertainment,Skating Rink,Farm,Café,Vietnamese Restaurant,Grocery Store,Gas Station,Furniture / Home Store,Fried Chicken Joint
